In [3]:
import math
import torch
import torchvision
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
import torch.nn.functional as F
from torch import nn
from tqdm import tqdm

In [8]:
class ConvBlock(nn.Module):
  def __init__(self, in_ch, out_ch, time_embed_dim):
    super().__init__()
    self.convs = nn.Sequential(
      nn.Conv2d(in_ch, out_ch, 3, padding=1),
      nn.BatchNorm2d(out_ch),
      nn.ReLU(),
      nn.Conv2d(out_ch, out_ch, 3, padding=1),
      nn.BatchNorm2d(out_ch),
      nn.ReLU()
    )
    self.mlp = nn.Sequential(
      nn.Linear(time_embed_dim, in_ch),
      nn.ReLU(),
      nn.Linear(in_ch, in_ch),
    )

  def forward(self, x, v):
    N, C, _, _ = x.shape
    v = self.mlp(v)
    v = v.view(N, C, 1, 1)
    y = self.convs(x + v)
    return y

def _pos_encoding(t, output_dim, device = "cpu"):
  D = output_dim
  v = torch.zeros(D, device=device)

  i = torch.arange(0, D, device=device)
  div_term = 10000 ** (i / D)

  v[0::2] = torch.sin(t / div_term[0::2])
  v[1::2] = torch.cos(t / div_term[1::2])
  return v

def pos_encoding(ts, output_dim, device = "cpu"):
  batch_size = len(ts)
  v = torch.zeros(batch_size, output_dim, device=device)
  for i in range(batch_size):
    v[i] = _pos_encoding(ts[i], output_dim, device=device)
  return v

class UNetCond(nn.Module):
	def __init__(self, in_ch=1, time_embed_dim=100, num_labels=None):
		super().__init__()
		self.time_embed_dim = time_embed_dim
		
		self.down1 = ConvBlock(in_ch, 64, time_embed_dim)
		self.down2 = ConvBlock(64, 128, time_embed_dim)
		self.bot1 = ConvBlock(128, 256, time_embed_dim)
		self.up2 = ConvBlock(128 + 256, 128, time_embed_dim)
		self.up1 = ConvBlock(128 + 64, 64, time_embed_dim)
		self.out = nn.Conv2d(64, in_ch, 1)

		self.maxpool = nn.MaxPool2d(2)
		self.upsample = nn.Upsample(scale_factor=2, mode='bilinear')
        
		if num_labels is not None:
			self.label_emb = nn.Embedding(num_labels, time_embed_dim)
               
	def forward(self, x, timesteps, labels=None):
		t = pos_encoding(timesteps, self.time_embed_dim)

		if labels is not None:
			t += self.label_emb(labels)
               
		x1 = self.down1(x, t)
		x = self.maxpool(x1)
		x2 = self.down2(x, t)
		x = self.maxpool(x2)
          
		x = self.bot1(x, t)
          
		x = self.upsample(x)
		x = torch.cat([x, x2], dim=1)
		x = self.up2(x, t)
		x = self.upsample(x)
		x = torch.cat([x, x1], dim=1)
		x = self.up1(x, t)
		x = self.out(x)
    
		return x
     

class Diffuser:
	def __init__(self, num_timesteps=1000, beta_start=0.0001, beta_end=0.02, device="cpu"):
		self.num_timesteps = num_timesteps
		self.device = device
		self.betas = torch.linspace(beta_start, beta_end, num_timesteps, device=device)
		self.alphas = 1 - self.betas
		self.alpha_bars = torch.cumprod(self.alphas, dim=0)

	def add_noise(self, x_0, t):
		assert (t >= 1).all() and (t <= self.num_timesteps).all()
		t_idx = t - 1

		alpha_bar = self.alpha_bars[t_idx]
		N = alpha_bar.size(0)
		alpha_bar = alpha_bar.view(N, 1, 1, 1)

		noise = torch.randn_like(x_0, device=self.device)
		x_t = torch.sqrt(alpha_bar) * x_0 + torch.sqrt(1 - alpha_bar) * noise
		return x_t, noise

	def denoise(self, model, x, t, labels):
		T = self.num_timesteps
		assert (t >= 1).all() and (t <= T).all()

		t_idx = t - 1
		alpha = self.alphas[t_idx]
		alpha_bar = self.alpha_bars[t_idx]
		alpha_bar_prev = self.alpha_bars[t_idx - 1]


		N = alpha.size(0)
		alpha = alpha.view(N, 1, 1, 1)
		alpha_bar = alpha_bar.view(N, 1, 1, 1)
		alpha_bar_prev = alpha_bar_prev.view(N, 1, 1, 1)

		model.eval()

		with torch.no_grad():
			eps = model(x, t, labels)
			model.train()

			noise = torch.randn_like(x, device=self.device)
			noise[t == 1] = 0

			mu = (x - ((1 - alpha) / torch.sqrt(1 - alpha_bar_prev)) * eps) / torch.sqrt(alpha)
			std = torch.sqrt((1-alpha) * (1 - alpha_bar_prev) / (1 - alpha_bar))
			return mu + noise * std

	def reverse_to_img(self, x):
		x = x * 255
		x = x.clamp(0, 255)
		x = x.to(torch.uint8)
		to_pil = transforms.ToPILImage()
		return to_pil(x)

	def sample(self, model, x_shape=(20, 1, 28, 28), labels=None):
		batch_size = x.shape[0]
		x = torch.randn(x_shape, device=self.device)
		
		if labels is None:
			labels = torch.randint(0, 10, (len(x),), device=self.device)

		for i in tqdm(range(self.num_timesteps, 0, -1)):
			t = torch.tensor([i] * batch_size, device=self.device, dtype=torch.long)
			x = self.denoise(model, x, t, labels)

		images = [self.reverse_to_img(x[i]) for i in range(batch_size)]
		
		return images, labels


In [11]:
img_size = 28
batch_size = 128
num_timesteps = 1000
epochs = 10
lr = 1e-3
device = "cuda" if torch.cuda.is_available() else "cpu"

def show_images(images, rows=2 ,cols=10):
  fig = plt.figure(figsize=(cols, rows))
  i = 0
  for r in range(rows):
    for c in range(cols):
      ax = fig.add_subplot(rows, cols, r * cols + c + 1)
      ax.imshow(images[i], cmap="gray")
      ax.title(labels[i].item())
      ax.axis("off")
      i += 1
  plt.show()


preprocess = transforms.ToTensor()
dataset = torchvision.datasets.MNIST(root="./data", transform=preprocess, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

diffuser = Diffuser(num_timesteps, device=device)
model = UNetCond(num_labels=10)
model.to(device)
optimizer = Adam(model.parameters(), lr=lr)

losses = []
for epoch in range(epochs):
  loss_sum = 0.0
  cnt = 0

  # エポックごとにデータ生成して結果を確認したい場合は以下のコメントアウトをとる
  images = diffuser.sample(model)
  show_images(images)
  
  for images, label in tqdm(dataloader):
    optimizer.zero_grad()
    x = images.to(device)
    labels = labels.to(device)
    t = torch.randint(1, num_timesteps + 1, (len(x),), device=device)

    x_noisy, noise = diffuser.add_noise(x, t)
    noise_pred = model(x_noisy, t, labels)
    loss = F.mse_loss(noise, noise_pred)

    loss.backward()
    optimizer.step()

    loss_sum += loss.item()
    cnt += 1



In [ ]:
import math
import torch
import torchvision
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
import torch.nn.functional as F
from torch import nn
from tqdm import tqdm

class ConvBlock(nn.Module):
  def __init__(self, in_ch, out_ch, time_embed_dim):
    super().__init__()
    self.convs = nn.Sequential(
      nn.Conv2d(in_ch, out_ch, 3, padding=1),
      nn.BatchNorm2d(out_ch),
      nn.ReLU(),
      nn.Conv2d(out_ch, out_ch, 3, padding=1),
      nn.BatchNorm2d(out_ch),
      nn.ReLU()
    )
    self.mlp = nn.Sequential(
      nn.Linear(time_embed_dim, in_ch),
      nn.ReLU(),
      nn.Linear(in_ch, in_ch),
    )

  def forward(self, x, v):
    N, C, _, _ = x.shape
    v = self.mlp(v)
    v = v.view(N, C, 1, 1)
    y = self.convs(x + v)
    return y

def _pos_encoding(t, output_dim, device = "cpu"):
  D = output_dim
  v = torch.zeros(D, device=device)

  i = torch.arange(0, D, device=device)
  div_term = 10000 ** (i / D)

  v[0::2] = torch.sin(t / div_term[0::2].to(device))
  v[1::2] = torch.cos(t / div_term[1::2].to(device))
  return v

def pos_encoding(ts, output_dim, device = "cpu"):
  batch_size = len(ts)
  v = torch.zeros(batch_size, output_dim, device=device)
  for i in range(batch_size):
    v[i] = _pos_encoding(ts[i], output_dim, device=device)
  return v

class UNetCond(nn.Module):
	def __init__(self, in_ch=1, time_embed_dim=100, num_labels=None):
		super().__init__()
		self.time_embed_dim = time_embed_dim
		
		self.down1 = ConvBlock(in_ch, 64, time_embed_dim)
		self.down2 = ConvBlock(64, 128, time_embed_dim)
		self.bot1 = ConvBlock(128, 256, time_embed_dim)
		self.up2 = ConvBlock(128 + 256, 128, time_embed_dim)
		self.up1 = ConvBlock(128 + 64, 64, time_embed_dim)
		self.out = nn.Conv2d(64, in_ch, 1)

		self.maxpool = nn.MaxPool2d(2)
		self.upsample = nn.Upsample(scale_factor=2, mode='bilinear')
        
		if num_labels is not None:
			self.label_emb = nn.Embedding(num_labels, time_embed_dim)
               
	def forward(self, x, timesteps, labels=None):
		t = pos_encoding(timesteps, self.time_embed_dim, device=x.device)

		if labels is not None:
			t += self.label_emb(labels)
               
		x1 = self.down1(x, t)
		x = self.maxpool(x1)
		x2 = self.down2(x, t)
		x = self.maxpool(x2)
          
		x = self.bot1(x, t)
          
		x = self.upsample(x)
		x = torch.cat([x, x2], dim=1)
		x = self.up2(x, t)
		x = self.upsample(x)
		x = torch.cat([x, x1], dim=1)
		x = self.up1(x, t)
		x = self.out(x)
    
		return x
     

class Diffuser:
	def __init__(self, num_timesteps=1000, beta_start=0.0001, beta_end=0.02, device="cpu"):
		self.num_timesteps = num_timesteps
		self.device = device
		self.betas = torch.linspace(beta_start, beta_end, num_timesteps, device=device)
		self.alphas = 1 - self.betas
		self.alpha_bars = torch.cumprod(self.alphas, dim=0)

	def add_noise(self, x_0, t):
		assert (t >= 1).all() and (t <= self.num_timesteps).all()
		t_idx = t - 1

		alpha_bar = self.alpha_bars[t_idx]
		N = alpha_bar.size(0)
		alpha_bar = alpha_bar.view(N, 1, 1, 1)

		noise = torch.randn_like(x_0, device=self.device)
		x_t = torch.sqrt(alpha_bar) * x_0 + torch.sqrt(1 - alpha_bar) * noise
		return x_t, noise

	def denoise(self, model, x, t, labels):
		T = self.num_timesteps
		assert (t >= 1).all() and (t <= T).all()

		t_idx = t - 1
		alpha = self.alphas[t_idx]
		alpha_bar = self.alpha_bars[t_idx]
		alpha_bar_prev = self.alpha_bars[t_idx - 1]


		N = alpha.size(0)
		alpha = alpha.view(N, 1, 1, 1)
		alpha_bar = alpha_bar.view(N, 1, 1, 1)
		alpha_bar_prev = alpha_bar_prev.view(N, 1, 1, 1)

		model.eval()

		with torch.no_grad():
			eps = model(x, t, labels)
			model.train()

			noise = torch.randn_like(x, device=self.device)
			noise[t == 1] = 0

			mu = (x - ((1 - alpha) / torch.sqrt(1 - alpha_bar_prev)) * eps) / torch.sqrt(alpha)
			std = torch.sqrt((1-alpha) * (1 - alpha_bar_prev) / (1 - alpha_bar))
			return mu + noise * std

	def reverse_to_img(self, x):
		x = x * 255
		x = x.clamp(0, 255)
		x = x.to(torch.uint8)
		to_pil = transforms.ToPILImage()
		return to_pil(x)

	def sample(self, model, x_shape=(20, 1, 28, 28), labels=None):
		batch_size = x_shape[0]
		x = torch.randn(x_shape, device=self.device)
		
		if labels is None:
			labels = torch.randint(0, 10, (len(x),), device=self.device)

		for i in tqdm(range(self.num_timesteps, 0, -1)):
			t = torch.tensor([i] * batch_size, device=self.device, dtype=torch.long)
			x = self.denoise(model, x, t, labels)

		images = [self.reverse_to_img(x[i]) for i in range(batch_size)]
		
		return images, labels


img_size = 28
batch_size = 128
num_timesteps = 1000
epochs = 10
lr = 1e-3
device = "cuda" if torch.cuda.is_available() else "cpu"

def show_images(images, labels, id, rows=2 ,cols=10):
  fig = plt.figure(figsize=(cols, rows))
  i = 0
  for r in range(rows):
    for c in range(cols):
      ax = fig.add_subplot(rows, cols, r * cols + c + 1)
      ax.imshow(images[i], cmap="gray")
      ax.set_title(labels[i].item())
      ax.axis("off")
      i += 1
  # plt.show()
  fig.tight_layout()
  plt.savefig(f"./image/{id}.png")


preprocess = transforms.ToTensor()
dataset = torchvision.datasets.MNIST(root="./data", transform=preprocess, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

diffuser = Diffuser(num_timesteps, device=device)
model = UNetCond(num_labels=10)
model.to(device)
optimizer = Adam(model.parameters(), lr=lr)

losses = []
for epoch in range(epochs):
  loss_sum = 0.0
  cnt = 0

  # エポックごとにデータ生成して結果を確認したい場合は以下のコメントアウトをとる
  images, label = diffuser.sample(model)
  show_images(images, label, id=epoch)
  
  for images, labels in tqdm(dataloader):
    optimizer.zero_grad()
    x = images.to(device)
    labels = labels.to(device)
    t = torch.randint(1, num_timesteps + 1, (len(x),), device=device)

    x_noisy, noise = diffuser.add_noise(x, t)
    noise_pred = model(x_noisy, t, labels)
    loss = F.mse_loss(noise, noise_pred)

    loss.backward()
    optimizer.step()

    loss_sum += loss.item()
    cnt += 1

